# Slicer Tutorial Brain MRI

This tutorial shows you how to use the functions from the module 'basic_functions'. Before you begin, <span style="color:red">**please ensure you have all the required modules downloaded from the requirements.txt file.** </span>

### Setup the slicer kernel

<ins>New functions used:</ins>
~~~
load_DICOM()
~~~
- loads in an image or segmentation into the scene stored as a DICOM file

In [3]:
import json
import logging.config
import logging.handlers
import sys
import os
import subprocess

# Add slicer_functions/ to the Python path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'slicer_functions')))

# Import the basic functions module
from basic_functions import *

In [4]:
# confugure logging
logger = logging.getLogger("slicer_tutorial")
config_file = os.path.join(os.getcwd(), 'logging_configs', 'config.json')
with open(config_file) as f:
    config = json.load(f)
logging.config.dictConfig(config)

In [5]:
# Load in the DICOM directory for the brain resection patient into slicer
current_dir = os.path.dirname(os.path.abspath('__file__'))
brain_resection_dicom_path = os.path.join(current_dir, '..', 'example_patients', 'cervical_cancer')

slicer.mrmlScene.Clear(0) # Clear the scene
load_DICOM(brain_resection_dicom_path)

2024-10-11 11:04:34,095 - basic_functions - INFO - Loading DICOM data from directory: /Users/marcus/Documents/GitHub/Slicer_tutorial/oop_testing/../example_patients/cervical_cancer
2024-10-11 11:04:34,096 - root - INFO - Switching to temporary DICOM database: /private/var/folders/ml/l95x031n6jbbl01q4b_391tr0000gn/T/Slicer-marcus/20241011_150434_TempDICOMDatabase
2024-10-11 11:04:34,191 - basic_functions - DEBUG - Importing DICOM data from directory: /Users/marcus/Documents/GitHub/Slicer_tutorial/oop_testing/../example_patients/cervical_cancer
2024-10-11 11:04:37,282 - basic_functions - DEBUG - Loading patient with UID: 1
2024-10-11 11:04:37,806 - root - DEBUG - DICOMM3DPluginClass : Caching files
2024-10-11 11:04:37,868 - root - DEBUG - DICOMM3DPluginClass : Caching files
2024-10-11 11:04:37,910 - root - DEBUG - DICOMM3DPluginClass : Caching files
2024-10-11 11:04:37,944 - root - DEBUG - DICOMM3DPluginClass : Caching files
2024-10-11 11:04:37,945 - root - DEBUG - DICOMM3DPluginClass : 

['vtkMRMLScalarVolumeNode1',
 'vtkMRMLScalarVolumeNode2',
 'vtkMRMLScalarVolumeNode3',
 'vtkMRMLScalarVolumeNode4',
 'vtkMRMLScalarVolumeNode5',
 'vtkMRMLScalarVolumeNode6',
 'vtkMRMLSegmentationNode1']

### Learn how to use the helper function

<ins>New functions used:</ins>
~~~
quick_visualize()
~~~
- displays axial, saggital, and coronal slices using matplotlib for quick verification onf numpy arrays

In [ ]:
help(quick_visualize)

### Access the data stored in the scene

<ins>New functions used:</ins>
~~~
get_ct_and_pet_volume_nodes()
~~~
- finds the ct and pet volume nodes and returns the node ID's in a tuple

In [6]:
# Get the volume nodes and set the PET cmap to heat
ct_node, pet_node = get_ct_and_pet_volume_nodes()

# Set the PET colormap to heat
petColor = slicer.mrmlScene.GetFirstNodeByName('PET-Heat')
pet_node.GetVolumeDisplayNode().SetAndObserveColorNodeID(petColor.GetID())
pet_node.GetVolumeDisplayNode().AutoWindowLevelOn()

#  set the CT colormap to abdomen window/level
slicer.modules.volumes.logic().ApplyVolumeDisplayPreset(ct_node.GetVolumeDisplayNode(), 'CT_ABDOMEN')


# Set the foreground and background of the slice viewer
slicer.util.setSliceViewerLayers(background=ct_node, foreground=pet_node, foregroundOpacity = 0.2)

In [7]:
# Get the segmentation node
segmentation_node = slicer.mrmlScene.GetFirstNodeByClass('vtkMRMLSegmentationNode')
segmentation = segmentation_node.GetSegmentation()

# Set the contours as BinaryLabelmap for manipulation
segmentation_node.CreateBinaryLabelmapRepresentation()
segmentation_node.SetSourceRepresentationToBinaryLabelmap()

True

In [9]:
# Get the names of all the segments in the segmentation node
all_segment_names = [segmentation.GetNthSegment(i).GetName() for i in range(segmentation.GetNumberOfSegments())]

# Get the volume nodes as numpy arrays
ct_numpy_array = slicer.util.arrayFromVolume(ct_node)
pet_numpy_array = slicer.util.arrayFromVolume(pet_node)

# Make a dictionary of the segment names, and their numpy array sampled from the ct
ct_segment_dict = {}
for segment_name in all_segment_names:
    segment_id = segmentation.GetSegmentIdBySegmentName(segment_name)
    if not segment_id is None:
        ct_segment_dict[segment_name] = slicer.util.arrayFromSegmentBinaryLabelmap(segmentation_node, segment_id, ct_node).astype(int)

### Let's calculate the max and average houndsfeild values of the feamur and lungs:

<ins>New functions used:</ins>
~~~
add_segmentation_array_to_node() 
~~~    
 
- add a new or modified segment from a numpy array to a segmentation node


In [ ]:
# Get the array of the left and right femur from the dictionary
left_femur = ct_segment_dict['femur_left']
right_femur = ct_segment_dict['femur_right']

left_femur_ct = np.multiply(left_femur, ct_numpy_array)
right_femur_ct = np.multiply(right_femur, ct_numpy_array)


# Get the max and average values of the left and right femur
logger.info(f"Left femur max: {np.max(left_femur_ct)}, Left femur average: {np.sum(left_femur_ct)/np.sum(left_femur)}")
logger.info(f"Right femur max: {np.max(right_femur_ct)}, Right femur average: {np.sum(right_femur_ct)/np.sum(right_femur)}")

In [ ]:
# Combine all the 5 lung lobes into one segment array
lung_array = np.zeros_like(ct_numpy_array)

for segment_name in all_segment_names:
    if "lung" in segment_name.lower():
        segment_id = segmentation.GetSegmentIdBySegmentName(segment_name)
        lung_array = bitwise_or_from_array(lung_array, ct_segment_dict[segment_name])

# Add the lung array to the dictionary and to slicer
ct_segment_dict['Lung'] = lung_array
add_segmentation_array_to_node(ct_segment_dict['Lung'], segmentation_node, 'Lung', ct_node)

lung_ct = np.multiply(lung_array, ct_numpy_array)

# Get the max and average values of the lung
logger.info(f"Lung max: {np.max(lung_ct)}, Lung average: {np.sum(lung_ct)/np.sum(lung_array)}")

### Create a new segmentation based on OTSU of the PET image
<ins>New functions used:</ins>
~~~
segmentation_by_auto_threshold()
~~~

- Creates a segmentation by an auto thresholding method

~~~
island_effect_segbment_editor()
~~~

- Implementation of the island effect in Slicer's Segment Editor module

~~~
dice_similarity()
~~~

- Computes the DICE similarity between two numpy arrays


In [ ]:
# Create a new blank segmentation node for the OTSU threshold based segmentation
OTSU_segmentation_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "OTSU_segmentation")
sh = slicer.mrmlScene.GetSubjectHierarchyNode() ### gets the subject hierarchy node to determine the parent of the new segmentation node
sh.SetItemParent(sh.GetItemByDataNode(OTSU_segmentation_node),  sh.GetItemParent(sh.GetItemByDataNode(ct_node))) ### sets the parent of the new segmentation node to the parent of the CT node
OTSU_segmentation_node.CreateDefaultDisplayNodes() ### creates the display nodes for the new segmentation node

# Set the geometry of the OTSU node to the PET node
OTSU_segmentation_node.SetReferenceImageGeometryParameterFromVolumeNode(pet_node)

# Create the OTSU segmentation
segmentation_by_auto_threshold("OTSU_segmentation", OTSU_segmentation_node, pet_node)

# Split up the segmentation into islands
islands_effect_segment_editor("OTSU_segmentation", OTSU_segmentation_node, pet_node, edit='SPLIT_ISLANDS_TO_SEGMENTS', minimumSize=1)

<span style="color:green">**Have a look at the created segmentations on slicer**</span>
- the segmentation OTSU_segmentation looks like the bladder, let's see how similar they are using the DICE similarity index

In [13]:
# Pull the numpy arrays of the OTSU segmentations
OTSU_segment_names = [OTSU_segmentation_node.GetSegmentation().GetNthSegment(i).GetName() for i in range(OTSU_segmentation_node.GetSegmentation().GetNumberOfSegments())]

OTSU_segment_dict = {}
for segment_name in OTSU_segment_names:
    segment_id = OTSU_segmentation_node.GetSegmentation().GetSegmentIdBySegmentName(segment_name)
    if not segment_id is None:
        OTSU_segment_dict[segment_name] = slicer.util.arrayFromSegmentBinaryLabelmap(OTSU_segmentation_node, segment_id, pet_node).astype(int)

# Get the bladder segmentation sampled from the PET image
pet_sampled_bladder = slicer.util.arrayFromSegmentBinaryLabelmap(segmentation_node, segmentation.GetSegmentIdBySegmentName("urinary_bladder"), pet_node).astype(int)

In [ ]:
# Perform the DICE similarity test
dice_result = dice_similarity(OTSU_segment_dict["OTSU_segmentation"], pet_sampled_bladder)

print(f"The dice results between the OTSU segmentation and the bladder is {dice_result}")

In [15]:
# The OTSU_segmentation and the bladder are similar! Let's remove the segmentation from the node
OTSU_segmentation_node.RemoveSegment("OTSU_segmentation")
OTSU_segment_dict.pop("OTSU_segmentation")

# The other threshold must be the tumor! Let's rename accordingly
OTSU_segmentation_node.GetSegmentation().GetSegment(OTSU_segmentation_node.GetSegmentation().GetSegmentIdBySegmentName("OTSU_segmentation_2")).SetName("Tumor")
OTSU_segment_dict['Tumor'] = OTSU_segment_dict["OTSU_segmentation_2"]
del OTSU_segment_dict["OTSU_segmentation_2"]

In [ ]:
# Add the tumor segmentation to the original segmentation node
add_segmentation_array_to_node(OTSU_segment_dict['Tumor'], segmentation_node, "Tumor", pet_node)

# Add the segmentation to the ct_segment_dict
ct_segment_dict['Tumor'] = slicer.util.arrayFromSegmentBinaryLabelmap(segmentation_node, segmentation.GetSegmentIdBySegmentName('Tumor'), ct_node).astype(int)
all_segment_names.append('Tumor')
del OTSU_segment_dict

# Remove the OTSU node
slicer.mrmlScene.RemoveNode(OTSU_segmentation_node)
del OTSU_segmentation_node

### Save a video of the segmentations and volumes/ segmentations as a nifti
<ins>New functions used:</ins>
~~~
sweep_screen_capture()
~~~

- Saves a sweep screen capture of the Slicer scene

~~~
save_image_from_DICOM_satabase()
~~~

- Saves a volume node from a DICOM into a nifti or nrrd

~~~
save_rtstructs_as_nii()
~~~

- Saves segmentations from a segmentation node into nifti format

In [17]:
# Create a save path
save_path = os.path.join(current_dir, "tutorial_saves") 
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
# Save a video

# Capture the video
sweep_screen_capture(ct_node, save_path, "tutorial_screen_capture", tupleOfSegmentationNodesToShow=(segmentation_node, ), foregroundImageNode=pet_node, foregroundOpacity=0.2) 

In [ ]:
# Save the files
save_image_from_scalar_volume_node(save_path, ct_node, additionalSaveInfo='CT') ## Saves the ct volume
save_image_from_scalar_volume_node(save_path, pet_node, additionalSaveInfo='PET') ## Saves the pet volume

segments_to_save = tuple(all_segment_names)
segmentation_node.SetName("Segmentations")
save_segmentations_to_files(segments_to_save, segmentation_node, ct_node, save_path, "nii")

In [ ]:
# View the video
subprocess.run(["start", os.path.join(save_path, "tutorial_screen_capture.mp4")], shell=True)

## Thank you for completing the tutorial!
If you have any questions or would like us to create a function for your use case, please contact Marcus Milantoni at mmilanto@uwo.ca or use the GitHub repository.

# Testing of OOP

In [10]:
help(ct_node)

Help on vtkMRMLScalarVolumeNode object:

class vtkMRMLScalarVolumeNode(vtkMRMLVolumeNode)
 |  vtkMRMLScalarVolumeNode - MRML node for representing a volume (image
 |  stack).
 |  
 |  Superclass: vtkMRMLVolumeNode
 |  
 |  Volume nodes describe data sets that can be thought of as stacks of
 |  2D images that form a 3D volume. Volume nodes contain only the image
 |  data, where it is store on disk and how to read the files is
 |  controlled by the volume storage node, how to render the data (window
 |  and level) is controlled by the volume display nodes. Image
 |  information is extracted from the image headers (if they exist) at
 |  the time the MRML file is generated. Consequently, MRML files isolate
 |  MRML browsers from understanding how to read the myriad of file
 |  formats for medical data.
 |  
 |  Method resolution order:
 |      vtkMRMLScalarVolumeNode
 |      vtkMRMLVolumeNode
 |      vtkMRMLDisplayableNode
 |      vtkMRMLTransformableNode
 |      vtkMRMLStorableNode
 |    

In [8]:
help(segmentation.RemoveSegment)

Help on built-in function RemoveSegment:

RemoveSegment(...) method of vtkSegmentationCorePython.vtkSegmentation instance
    RemoveSegment(self, segmentId:str) -> None
    C++: void RemoveSegment(std::string segmentId)
    RemoveSegment(self, segment:vtkSegment) -> None
    C++: void RemoveSegment(vtkSegment *segment)
    
    Remove a segment by ID
    \param segmentId Identifier of the segment to remove from the
        segmentation

